## Combined models
1) rf-benign-specialize
2) rf-general
3) rf-lexical

In [1]:
import os
import pickle

from collections import defaultdict
import numpy as np
import pandas as pd 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler

In [2]:
def classification_type(type):
    '''
    Convert classification type into values:
        1) Benign = 0
        2) Defacement = 1
        3) Phishing = 2
        4) Malware = 3    
    '''
    if type == "benign":
        return 0
    elif type == "defacement":
        return 1
    elif type == "phishing":
        return 2
    elif type == "malware":
        return 3
    else:
        print(f"Unable to find proper type: {type}")


def calc_FNR_accuracy(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    for label_class in range(4):
        FN = sum(conf_matrix[label_class][i] for i in range(len(conf_matrix)) if i != label_class)  
        
        TP = conf_matrix[label_class][label_class]  
        
        TN = np.sum(np.delete(np.delete(conf_matrix, label_class, axis=0), label_class, axis=1))
        
        accuracy = (TP + TN) / np.sum(conf_matrix)
        print("Accuracy for class", label_class, ":", accuracy)

        FNR = FN / (FN + TP) if (FN + TP) > 0 else -1
        print("FNR for class", label_class, ":", FNR)

In [3]:
# Load models
model_files = [file for file in os.listdir(os.getcwd()) if file.endswith(".pkl")]
models = []
for file in model_files:
    with open(file, "rb") as file:
        models.append((file.name, pickle.load(file)))

In [4]:
# Data for training ensemble model
training_dataset = {'rf-benign-specialize.pkl': 'rf-benign-specialize-features.csv',
                    'rf-general.pkl': 'rf-general-features.csv',
                    'rf-lexical.pkl': 'rf-lexical-features.csv'}

df = pd.read_csv('../datasets/malicious_phish.csv')
y = df.iloc[:, 1]
y = y.apply(classification_type)
y = y.values.ravel()

In [5]:
testing_ensemble_X = {} # test size 0.2 values <model_name: (X_test, scaler)>
testing_ensemble_y = None # test size 0.2 values
scalers = {} # stores scaler models <model_name: scaler>

# Prepare predictions from models
for i in range(len(models)):
    model_name, model = models[i]
    data = pd.read_csv(training_dataset[model_name], header=None, skiprows=1)
    X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=69)
    
    scaler = MaxAbsScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    if testing_ensemble_y is None:
        testing_ensemble_y = y_test

    testing_ensemble_X[model_name] = X_test
    scalers[model_name] = scaler

### Test on 0.2 validation split initial dataset

In [6]:
y_ensemble_pred = None # holds predictions of all models
model_col = {} # Takes note of which column the model predictions are in <index, model_name>
model_col_rev = {} # <model_name, index>

for i in range(len(models)):
    model_name, model = models[i]
    X_test = testing_ensemble_X[model_name] # Already scaled
    
    y_pred = model.predict(X_test)
    if y_ensemble_pred is None:
        y_ensemble_pred = np.empty((y_pred.shape[0], 3))

    y_ensemble_pred[:, i] = y_pred
    model_col[i] = model_name
    model_col_rev[model_name] = i

# Look at priority list
pred_combined_y = y_ensemble_pred[:, model_col_rev["rf-general.pkl"]]

row = 0
for entry in y_ensemble_pred:
    index = model_col_rev["rf-benign-specialize.pkl"]
    if entry[index] == 0:
        pred_combined_y[row] = 0
    row += 1

/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [7]:
precision = precision_score(testing_ensemble_y, pred_combined_y, average='weighted')
recall = recall_score(testing_ensemble_y, pred_combined_y, average='weighted')
f1 = f1_score(testing_ensemble_y, pred_combined_y, average='weighted')
val_accuracy = accuracy_score(testing_ensemble_y, pred_combined_y)
print(f'OVERALL: Accuracy: {val_accuracy:.8f}, Precision: {precision:.8f}, Recall: {recall:.8f}, F1 Score: {f1:.8f}')

class_test_precision, class_test_recall, class_test_f1, class_ = precision_recall_fscore_support(testing_ensemble_y, pred_combined_y)
for i in range(4):
    print(f'Class {i}:\tTest Precision: {class_test_precision[i]:.8f},\tTest Recall: {class_test_recall[i]:.8f},\tTest f1: {class_test_f1[i]:.8f}')
calc_FNR_accuracy(testing_ensemble_y, pred_combined_y)

OVERALL: Accuracy: 0.85631032, Precision: 0.87466606, Recall: 0.85631032, F1 Score: 0.84122139
Class 0:	Test Precision: 0.82608317,	Test Recall: 0.99705779,	Test f1: 0.90355346
Class 1:	Test Precision: 0.98200969,	Test Recall: 0.66263748,	Test f1: 0.79131431
Class 2:	Test Precision: 0.94384932,	Test Recall: 0.42833485,	Test f1: 0.58925553
Class 3:	Test Precision: 0.99552239,	Test Recall: 0.80922050,	Test f1: 0.89275556
Accuracy for class 0 : 0.8600188883514155
FNR for class 0 : 0.0029422066549912433
Accuracy for class 1 : 0.9482720229731494
FNR for class 1 : 0.33736252334509237
Accuracy for class 2 : 0.914173173934075
FNR for class 2 : 0.5716651530530477
Accuracy for class 3 : 0.9901565583273827
FNR for class 3 : 0.1907794965119806


### Test on phishing dataset

In [8]:
training_dataset_phishing = {'rf-benign-specialize.pkl': 'rf-benign-specialize-test-phishing.csv',
                    'rf-general.pkl': 'rf-general-test-phishing.csv',
                    'rf-lexical.pkl': 'rf-lexical-test-phishing.csv'}

input_ensemble_phishing = None

# Prepare predictions from models
for i in range(len(models)):
    model_name, model = models[i]
    X_test = pd.read_csv(training_dataset_phishing[model_name], header=None, skiprows=1)
    X_test = scalers[model_name].transform(X_test)

    y_pred = model.predict(X_test) # prediction for model
    if input_ensemble_phishing is None:
        input_ensemble_phishing = np.empty((y_pred.shape[0], 3))

    input_ensemble_phishing[:, i] = y_pred

/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [9]:
# Phishing is label 2.
y_phishing = np.full((input_ensemble_phishing.shape[0],), 2)
pred_combined_y_phishing = input_ensemble_phishing[:, model_col_rev["rf-general.pkl"]]

row = 0
for entry in input_ensemble_phishing:
    index = model_col_rev["rf-benign-specialize.pkl"]
    if entry[index] == 0:
        pred_combined_y_phishing[row] = 0
    row += 1

In [10]:
precision = precision_score(y_phishing, pred_combined_y_phishing, average='weighted')
recall = recall_score(y_phishing, pred_combined_y_phishing, average='weighted')
f1 = f1_score(y_phishing, pred_combined_y_phishing, average='weighted')
val_accuracy = accuracy_score(y_phishing, pred_combined_y_phishing)
print(f'OVERALL: Accuracy: {val_accuracy:.8f}, Precision: {precision:.8f}, Recall: {recall:.8f}, F1 Score: {f1:.8f}')

class_test_precision, class_test_recall, class_test_f1, class_ = precision_recall_fscore_support(y_phishing, pred_combined_y_phishing)
for i in range(4):
    print(f'Class {i}:\tTest Precision: {class_test_precision[i]:.8f},\tTest Recall: {class_test_recall[i]:.8f},\tTest f1: {class_test_f1[i]:.8f}')
calc_FNR_accuracy(y_phishing, pred_combined_y_phishing)

OVERALL: Accuracy: 0.55475524, Precision: 1.00000000, Recall: 0.55475524, F1 Score: 0.71362389
Class 0:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Class 1:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Class 2:	Test Precision: 1.00000000,	Test Recall: 0.55475524,	Test f1: 0.71362389
Class 3:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Accuracy for class 0 : 0.5888127930906515
FNR for class 0 : -1
Accuracy for class 1 : 0.9685739347525387
FNR for class 1 : -1
Accuracy for class 2 : 0.5547552376775412
FNR for class 2 : 0.44524476232245874
Accuracy for class 3 : 0.9973685098343511
FNR for class 3 : -1


/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Test on benign dataset

In [11]:
training_dataset_benign = {'rf-benign-specialize.pkl': 'rf-benign-specialize-test-benign.csv',
                    'rf-general.pkl': 'rf-general-test-benign.csv',
                    'rf-lexical.pkl': 'rf-lexical-test-benign.csv'}

input_ensemble_benign = None

# Prepare predictions from models
for i in range(len(models)):
    model_name, model = models[i]
    X_test = pd.read_csv(training_dataset_benign[model_name], header=None, skiprows=1)
    X_test = scalers[model_name].transform(X_test)

    y_pred = model.predict(X_test) # prediction for model
    if input_ensemble_benign is None:
        input_ensemble_benign = np.empty((y_pred.shape[0], 3))

    input_ensemble_benign[:, i] = y_pred

/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [12]:
# Benign is label 0.
y_benign = np.full((input_ensemble_benign.shape[0],), 0)

pred_combined_y_benign = input_ensemble_benign[:, model_col_rev["rf-general.pkl"]]

row = 0
for entry in input_ensemble_benign:
    index = model_col_rev["rf-benign-specialize.pkl"]
    if entry[index] == 0:
        pred_combined_y_benign[row] = 0
    row += 1

In [13]:
precision = precision_score(y_benign, pred_combined_y_benign, average='weighted')
recall = recall_score(y_benign, pred_combined_y_benign, average='weighted')
f1 = f1_score(y_benign, pred_combined_y_benign, average='weighted')
val_accuracy = accuracy_score(y_benign, pred_combined_y_benign)
print(f'OVERALL: Accuracy: {val_accuracy:.8f}, Precision: {precision:.8f}, Recall: {recall:.8f}, F1 Score: {f1:.8f}')

class_test_precision, class_test_recall, class_test_f1, class_ = precision_recall_fscore_support(y_benign, pred_combined_y_benign)
for i in range(4):
    print(f'Class {i}:\tTest Precision: {class_test_precision[i]:.8f},\tTest Recall: {class_test_recall[i]:.8f},\tTest f1: {class_test_f1[i]:.8f}')
calc_FNR_accuracy(y_benign, pred_combined_y_benign)

OVERALL: Accuracy: 0.81864302, Precision: 1.00000000, Recall: 0.81864302, F1 Score: 0.90027896
Class 0:	Test Precision: 1.00000000,	Test Recall: 0.81864302,	Test f1: 0.90027896
Class 1:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Class 2:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Class 3:	Test Precision: 0.00000000,	Test Recall: 0.00000000,	Test f1: 0.00000000
Accuracy for class 0 : 0.8186430187020229
FNR for class 0 : 0.1813569812979771
Accuracy for class 1 : 0.9153839034181953
FNR for class 1 : -1
Accuracy for class 2 : 0.9034471189166363
FNR for class 2 : -1
Accuracy for class 3 : 0.9998119963671913
FNR for class 3 : -1


/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/asonata/anaconda3/envs/cs3264/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
